# Stoneburner, Kurt
- ## DSC 540 - Week XX/XX
- ## Chapter X, Exercise X


### Wrangling the BoingBoing Candy Survey
This is fun. Boing Boing has been in my daily reading rotation for a longtime. So of course I have to work with the data. It doesn't hit the same notes as the symphony, but the data is sweet.

In [7]:
import os
import sys
# //*** Imports and Load Data
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz

In [2]:
#There are three years of the Candy surveys, import each one
df_2015 = pd.read_excel("z_sup_wk07_08_candy2015.xlsx")
df_2016 = pd.read_excel("z_sup_wk07_08_candy2016.xlsx")
df_2017 = pd.read_excel("z_sup_wk07_08_candy2017.xlsx")


In [3]:
#//*** It's a little early for Merging, But let's look at the columns to determine which ones have candy types.
#//*** No sense in cleaning columns we are not going to use.
#//*** 2015 - Candy names start at the beginning of the column name and are wrapped in brackets

#//*** Get Candy Columns for 2015. Use Regex to to match the start of the string, any character then Bracket.
#//*** This eliminates the degrees of separation quetions.
#/**** Trim the leading space and brackets while we are at it. 
cols_2015 = df_2015.columns [ df_2015.columns.str.match('^.\[')]

#//*** Generate a cleaner list of column names. The clean name will be used to concatenate the dataframes
cols_2015_clean = df_2015.columns [ df_2015.columns.str.match('^.\[')].str.replace('^.',"").str.replace('[',"").str.replace(']',"")
#//*** This is a good starting point for 2015 columns
#print(cols_2015)
print(f"{cols_2015_clean}")

Index(['Butterfinger', '100 Grand Bar',
       'Anonymous brown globs that come in black and orange wrappers',
       'Any full-sized candy bar', 'Black Jacks', 'Bonkers', 'Bottle Caps',
       'Box’o’ Raisins', 'Brach products (not including candy corn)',
       'Bubble Gum', 'Cadbury Creme Eggs', 'Candy Corn',
       'Vials of pure high fructose corn syrup, for main-lining into your vein',
       'Candy that is clearly just the stuff given out for free at restaurants',
       'Cash, or other forms of legal tender', 'Chiclets', 'Caramellos',
       'Snickers', 'Dark Chocolate Hershey', 'Dental paraphenalia', 'Dots',
       'Fuzzy Peaches', 'Generic Brand Acetaminophen', 'Glow sticks',
       'Broken glow stick', 'Goo Goo Clusters', 'Good N' Plenty',
       'Gum from baseball cards', 'Gummy Bears straight up',
       'Creepy Religious comics/Chick Tracts', 'Healthy Fruit', 'Heath Bar',
       'Hershey’s Kissables', 'Hershey’s Milk Chocolate',
       'Hugs (actual physical hugs)', 'Joll

In [4]:
#//*** Get Candy Columns for 2016. These questions are structured using the same methods as 2015.
cols_2016 = df_2016.columns [ df_2016.columns.str.match('^.\[')]
#.str.replace('^.',"").str.replace('[',"").str.replace(']',"")


#//*** Generate a cleaner list of column names. The clean name will be used to concatenate the dataframes
cols_2016_clean = df_2016.columns [ df_2016.columns.str.match('^.\[')].str.replace('^.',"").str.replace('[',"").str.replace(']',"")

print(cols_2016_clean)

Index(['100 Grand Bar',
       'Anonymous brown globs that come in black and orange wrappers',
       'Any full-sized candy bar', 'Black Jacks', 'Bonkers (the candy)',
       'Bonkers (the board game)', 'Bottle Caps', 'Box'o'Raisins',
       'Broken glow stick', 'Butterfinger',
       ...
       'Tolberone something or other', 'Trail Mix', 'Twix',
       'Vials of pure high fructose corn syrup, for main-lining into your vein',
       'Vicodin', 'Whatchamacallit Bars', 'White Bread',
       'Whole Wheat anything', 'York Peppermint Patties',
       'York Peppermint Patties Ignore'],
      dtype='object', length=101)


In [5]:
#//*** Get Candy Columns for 2017. All Candy questions Begin with Q6 in the Column Name.
cols_2017 = df_2017.columns [ df_2017.columns.str.match('^Q6')]
#.str.replace('^Q6 \| ','')
cols_2017_clean = df_2017.columns [ df_2017.columns.str.match('^Q6')].str.replace('^Q6 \| ','')

print(cols_2017_clean)

Index(['100 Grand Bar',
       'Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes)',
       'Any full-sized candy bar', 'Black Jacks', 'Bonkers (the candy)',
       'Bonkers (the board game)', 'Bottle Caps', 'Box'o'Raisins',
       'Broken glow stick', 'Butterfinger',
       ...
       'Three Musketeers', 'Tolberone something or other', 'Trail Mix', 'Twix',
       'Vials of pure high fructose corn syrup, for main-lining into your vein',
       'Vicodin', 'Whatchamacallit Bars', 'White Bread',
       'Whole Wheat anything', 'York Peppermint Patties'],
      dtype='object', length=103)


In [13]:
#//**** Get the differences between two lists. Code which I liberally borrowed from the geeksforgeeks.org
#//*** Determine the differences in candy values

print("Candy Column Lengths")
print(f"2015: {len(cols_2015_clean)} - 2016: {len(cols_2016_clean)} - 2017: {len(cols_2017_clean)}")

               
                 
#//*** These are not equal, we need to remove the items not found in all three dataframes. Likely due to
#//*** Fun statistical questions like: Vicodin. Which are likely used to reduce testing bias (or the researchers)
#//*** have an awesome sense of humour or both).

for x in cols_2015_clean:
    
    best_y_score = 0
    best_y_val = ""
    
    for y in cols_2016_clean:
        score = fuzz.ratio(x,y)
        if score > best_y_score:
            best_y_val = y
            best_y_score = score
    if best_y_score < 100:
        
        print(f"1st: {best_y_score} : {x} : {best_y_val} ")
        best_y_score = 0
        best_y_val = ""
        
        for y in cols_2016_clean:
            score = fuzz.partial_ratio(x,y)
            if score > best_y_score:
                best_y_val = y
                best_y_score = score
        
        if best_y_score < 100:
            print(f"2nd: {best_y_score} : {x} : {best_y_val} ")


Candy Column Lengths
2015: 95 - 2016: 101 - 2017: 103
1st: 67 : Bonkers : Snickers 
1st: 81 : Box’o’ Raisins : Box'o'Raisins 
2nd: 77 : Box’o’ Raisins : Box'o'Raisins 
1st: 43 : Brach products (not including candy corn) : Bonkers (the candy) 
2nd: 80 : Brach products (not including candy corn) : Candy Corn 
1st: 40 : Bubble Gum : Blue M&M's 
2nd: 50 : Bubble Gum : Reese’s Peanut Butter Cups 
1st: 61 : Dark Chocolate Hershey : Hershey's Dark Chocolate 
2nd: 78 : Dark Chocolate Hershey : Hershey's Dark Chocolate 
1st: 86 : Hershey’s Kissables : Hershey's Kisses 
2nd: 81 : Hershey’s Kissables : Hershey's Kisses 
1st: 45 : Lapel Pins : Senior Mints 
2nd: 50 : Lapel Pins : Dental paraphenalia 
1st: 57 : Licorice : Licorice (not black) 
1st: 97 : JoyJoy (Mit Iodine) : JoyJoy (Mit Iodine!) 
2nd: 95 : JoyJoy (Mit Iodine) : JoyJoy (Mit Iodine!) 
1st: 47 : Runts : Junior Mints 
2nd: 60 : Runts : Candy that is clearly just the stuff given out for free at restaurants 
1st: 64 : Mint Leaves : Mint 

In [ ]:
1

In [ ]:
#//*** Build New Dataframes with just the columns we are going to combine.
concat_df_2015 = pd.DataFrame()
concat_df_2016 = pd.DataFrame()
concat_df_2017 = pd.DataFrame()

#//*** Add the Age Columns Manually
concat_df_2015['age'] = df_2015['How old are you?']
concat_df_2016['age'] = df_2016['How old are you?']
concat_df_2017['age'] = df_2017['Q3: AGE']


## Chapter 7 - Task1 ###
**Filter Out Missing Values**

In the Drop rows where no Candy values were filled out

In [ ]:
#//*** Drop rows where None of the Candy columns filled out
print(f"2015 length  before cleaning: {len(df_2015)}")
print(f"2016 length before cleaning: {len(df_2016)}")
print(f"2017 length before cleaning: {len(df_2017)}")


df_2015.dropna(axis=0,how='all',subset=cols_2015,inplace=True)
df_2016.dropna(axis=0,how='all',subset=cols_2016,inplace=True)
df_2017.dropna(axis=0,how='all',subset=cols_2017,inplace=True)

print(f"2015 length after cleaning: {len(df_2015)}")
print(f"2016 length after cleaning: {len(df_2016)}")
print(f"2017 length after cleaning: {len(df_2017)}")


## Chapter 7 - Task 2###
**Replace Values**

In many cases usere did not fill out all the form data. If we dropped these values, there wouldn't be much of a survey. Replace NaN with 0. Which will be used as a categorical for not answered.

In [ ]:
#//*** Replace NaN with 0 in 2015 Data
df_2015.fillna(value=0,axis=0,inplace=True)
print(df_2015.head(10))

#//*** Replace NaN with 0 in 2016 Data
df_2016.fillna(value=0,axis=0,inplace=True)

#//*** Replace NaN with 0 in 2017 Data
df_2017.fillna(value=0,axis=0,inplace=True)

In [ ]:
print(df_2015)

## Chapter 7 - Task 3###
**Transform Data**

Convert the categorical descriptions from Despair and Joy to 1 and 2

In [ ]:
#//*** Check for unique values in the candy columsn
#//*** Looking for miscodes or irregularities
print(pd.unique(df_2015[cols_2015].values.ravel()))
print(pd.unique(df_2016[list(cols_2016)].values.ravel()))
print(pd.unique(df_2017[list(cols_2017)].values.ravel()))


#for x in cols_2016:
#    print(df_2016[x].iloc[0])

In [ ]:
#//*** Apply a map to e
def remap_df(input_df, input_cols, input_map):
    
    for col in input_cols:
        
        input_df[col] = input_df[col].map(input_map, na_action='ignore')
    

#//**************************************************************************
#//*** Create Map based on Unique Values
#//*** Replace Joy with 1, because joy should always go first
#//*** Replace Despair with 2, because despair is greater than joy
#//*** Replace Meh with 3, because apathy conquers all
#//**************************************************************************

remap_ratings = {'JOY' : 1, 'DESPAIR' : 2, 'MEH' : 3, 0 : 0}

#//*** Mapping only applies to a series.
#//*** Loop through the dataframes and apply the mapping to each column.
#//*** This feels like a good use for a generator or Lambda expression.
#//*** But I'm not that cool today.
#//*** Do it with Loops because ... Loops

#//*** Remaps can only be applied once. Adding a conditional to run if JOY exist as a unique value
#//*** This speeds up coding / debugging due to the reusable nature of iPython
#// Remap 2015
if 'JOY' in list(pd.unique(df_2015[cols_2015].values.ravel())):
    remap_df(df_2015, cols_2015, remap_ratings)

#// Remap 2016
if 'JOY' in list(pd.unique(df_2016[cols_2016].values.ravel())):
    remap_df(df_2016, cols_2016, remap_ratings)

#// Remap 2017    
if 'JOY' in list(pd.unique(df_2017[cols_2017].values.ravel())):
    remap_df(df_2017, cols_2017, remap_ratings)


    
#//*** Visually Check our Work   
print(df_2015.head(5))
print(df_2016.head(5))
print(df_2017.head(5))
